# Lab:  Feature Selection for Linear Regression for Student Performance Data  

In this lab we use the UCI dataset of `Student Performance` to use linear regression with LASSO regularization. We will also look at some Feature Selection methods. The dataset is about student achievement in secondary education of two Portuguese schools. The target variable is the student's grade in their Mathematics exam and there are many features such as  demographic (address), social (family, age, sex, etc) and school related (schoolName, study time etc) features. So, we will try to predict the student's grades based on their background.

This lab has the following objectives

1. Learn about converting the categorical dataset to numerical values.
2. Perform LASSO regression and compare the results with simple linear regression.
3. Visualize the features obtained by LASSO and the LASSO path.
4. Learn another technique for feature selection.

## Loading the data
The dataset is available at 
<a href = "https://archive.ics.uci.edu/ml/datasets/Student+Performance">
P. Cortez and A. Silva. Using Data Mining to Predict Secondary School Student Performance. In A. Brito and J. Teixeira Eds., Proceedings of 5th FUture BUsiness TEChnology Conference (FUBUTEC 2008) pp. 5-12, Porto, Portugal, April, 2008, EUROSIS, ISBN 978-9077381-39-7. </a>

You need to download the Data Folder which is a `student.zip` file. It contains `student-mat.csv` file which we will use in this lab. You should go through the website to understand the meaning of each feature in the dataset, to be able to interpret your results.

We start with loading the basic packages.

In [1]:
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

Now, use `pd.read_csv(...)` to load the `student-mat.csv` file. Also, print the first 6 samples of dataframe `df`.

In [2]:
#TODO
df = pd.read_csv('student/student-mat.csv', sep = ';')
df.head(6)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
5,GP,M,16,U,LE3,T,4,3,services,other,...,5,4,2,1,2,5,10,15,15,15


You can see that the dataset contains a mixture of numerical and categorial features. For our analysis we can convert the categories to a numerical value. We can use two techniques-

1. **One-Hot Coding**: Create K new binary features for each categorical feature with K categories.
2. **Label Encoder**: Map categorical values of a feature to numericals using whole numbers (0,1,2,...).

We first look at the datatype of each features. Use the command `df.dtypes` and display the results.


In [3]:
df.dtypes

school        object
sex           object
age            int64
address       object
famsize       object
Pstatus       object
Medu           int64
Fedu           int64
Mjob          object
Fjob          object
reason        object
guardian      object
traveltime     int64
studytime      int64
failures       int64
schoolsup     object
famsup        object
paid          object
activities    object
nursery       object
higher        object
internet      object
romantic      object
famrel         int64
freetime       int64
goout          int64
Dalc           int64
Walc           int64
health         int64
absences       int64
G1             int64
G2             int64
G3             int64
dtype: object

Some of the features are of datatype **object**. Use the `select_dtypes` method in Pandas DataFrame to identify the categorical features (features of datatype `object`) and save the name of those features into a list `categorical_features`. Print this list. You should get these set of features: ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']

In [4]:
#TODO

categorical_features = np.array(df.select_dtypes(include=object).axes[1]).tolist()
print(categorical_features)

['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']


## One-Hot Coding and Label Encoder
For a categorical feature with more than two categories, we should use one-hot-coding (OHC) to convert it to binary features. However, for a categorical feature with only two categories, we should apply Label Encoder.  We first list the features that need OHC and those that need Label Encoder.





In [5]:
ohc_category = ['Mjob','Fjob','reason','guardian']
le_category = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'schoolsup', 'famsup', 'paid', 'activities',
       'nursery', 'higher', 'internet', 'romantic']

## One-Hot Coding
We first use One-Hot Coding to all categorical features. Pandas has a method called `get_dummies()` to do the job. It's interesting that this method is called `get_dummies` because it generates new dummy features corresponding to each categories. Find a new dataframe df_ohc which replace those features in the ohc_category by one-hot coding (apply get_dummies on columns in ohc_category). Also print the first 6 samples of the new dataframe `df_ohc`, and observe and comment on how each categorical feature is converted to multiple binary numerical features.

In [12]:
#TODO
df_ohc = df.drop(ohc_category, axis=1).join(pd.get_dummies(df[ohc_category]))
df_ohc.head(6)


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other
0,GP,F,18,U,GT3,A,4,4,2,2,...,0,0,1,1,0,0,0,0,1,0
1,GP,F,17,U,GT3,T,1,1,1,2,...,1,0,0,1,0,0,0,1,0,0
2,GP,F,15,U,LE3,T,1,1,1,2,...,1,0,0,0,0,1,0,0,1,0
3,GP,F,15,U,GT3,T,4,2,1,3,...,0,1,0,0,1,0,0,0,1,0
4,GP,F,16,U,GT3,T,3,3,1,2,...,1,0,0,0,1,0,0,1,0,0
5,GP,M,16,U,LE3,T,4,3,1,2,...,1,0,0,0,0,0,1,0,1,0


## Linear Encoder
Now we further convert those in the `df_ohc` data frame that are in the `le_category` using Label Encoder. Find a new dataframe `df_le` which is a copy of dataframe `df_ohc` except that all the binary categorial features are encoded to a numerical value of 0 or 1. You should use the `fit_transform()` method of the `LabelEncoder()`. Print first 6 lines of `df_le`, and make sure the entries in the final data frame are all properly encoded into numerical features.


In [38]:
from sklearn.preprocessing import LabelEncoder

#TODO

df_le = df_ohc.copy()

# Hint: Now use a for loop over the elements in `le_category` and update df_le
for cat in le_category:
    le = LabelEncoder()
    le.fit(np.array(df_le[cat]))
    df_le[cat] = le.transform(df_le[cat])
df_le.head(6)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other
0,0,0,18,1,0,0,4,4,2,2,...,0,0,1,1,0,0,0,0,1,0
1,0,0,17,1,0,1,1,1,1,2,...,1,0,0,1,0,0,0,1,0,0
2,0,0,15,1,1,1,1,1,1,2,...,1,0,0,0,0,1,0,0,1,0
3,0,0,15,1,0,1,4,2,1,3,...,0,1,0,0,1,0,0,0,1,0
4,0,0,16,1,0,1,3,3,1,2,...,1,0,0,0,1,0,0,1,0,0
5,0,1,16,1,1,1,4,3,1,2,...,1,0,0,0,0,0,1,0,1,0


The dataset has three targets namely G1, G2, and G3 which represents the grades in midterm1, midterm2 and final exams respectively. These variables are highly correlated with each other and therefore, if we use G3 as out target, it is not interesting to include G1 and G2 to our features. For our exercise, we will drop G1,G2, and G3 from the feature list, and use G1 as the target. You could try to use G2 or G3 as the target as well and see what happens, but submit the results with target 'G1' only.  

In [39]:
X_df = df_le.drop(['G1','G2','G3'],axis=1)
col_names = X_df.columns
X = np.array(X_df)
y = np.array(df_le['G1'])

If there are `nsamples` number of samples and `nfeatures` number of features, use the `shape` method to find them and print their values.

In [44]:
# TODO 
(nsamples, nfeatures) = X.shape
print("There are %d number of samples and %d number of features." % (nsamples, nfeatures))

There are 395 number of samples and 43 number of features.


## Using Linear Regression

Train a linear model using half of the samples and test the trained model using the other half samples. Print the Normalized train and test RSS.

In [ ]:
from sklearn import linear_model

# TODO
ns_train = nsamples // 2

You should observe that the normalized training RSS is reasonably small, but the testing error is large! One way to understand why this is the case is by printing the coeffcient values. Use the
`coef_` method of model `regr` to get the regression coefficients. Print the coefficients in decreasing order of their magnitudes.

In [ ]:
#TODO


#### Question:
Can you explain in one or two sentences below why the linear regression gives a large test error?

#### Type Answer Here:



## Using LASSO regression

Now let us try to use LASSO regression to select the optimal features. Note that it is extremely important to normalize (standardize) the data for the regularization methods. 

First use the `preprocessing.scale` method to standardize the data matrix `X` and target `y`.  Store the standardized values in `Xs` and `ys` respectively.    For this data, the `scale` routine may throw a warning that you are converting data types.  That is fine.

In [ ]:
from sklearn import preprocessing

# TODO


Now, use the LASSO method to fit a model.  Use cross validation to select the regularization level `alpha`.  Use 100 `alpha` values logarithmically spaced from `1e-3` to `10`, and use 10 fold cross validation.  Store the test RSS in a matrix `RSSts` with rows correponding to different `alpha` values and columns corresponding to different cross validation folds. 

In [ ]:
import  sklearn.model_selection 

#TODO


Determine the RSS mean and standard error corresponding to different alpha and plot the mean RSS with error bar as a function of alpha. Label the axis.

In [ ]:
# TODO

# RSS_mean = 
# RSS_std = 
# plt.errorbar(...)


Find the optimal `alpha` and the mean test RSS at this optimal point using the one standard error rule.

In [ ]:
# TO DO


## LASSO path

To further illustrate the effect of regularization, we conclude by drawing the *LASSO path*.  This is simply a plot of the coefficients as a function of the regularization `alpha`. The path demonstrates the effect of regularization well. Use the `lasso_path` method to obtain all the coefficients for the given range of alphas and plot the LASSO path. Also draw a vertical line at optimal value of alpha.

In [ ]:
# TODO


### Non-zero LASSO Coefficients
Using the above coefficients, Plot the number of non-zero coefficients vs alpha. Also draw a vertical line at optimal value of alpha. You can assume any coefficients with magnitude <= 0.001 to be zero.

In [ ]:
# TODO


Let us now find out with this optimal alpha, what coefficients are nonzero. Let us consider any coefficients with absoluate value <= 0.001 as zero. You need to first do a model fit using the entire data (`Xs`, `ys`) to find the model coefficients. Then determine and **print** the number of nonzeros (Call it `nfea1`). Finally, **print** the corresponding feature name (Hint: use the list `col_names` obtained previously) with their corresponding coefficient in the order with decreasing coefficient magnitudes.

In [ ]:
#TO DO

#First do a model fit using alpha_opt

# model.alpha = alpha_opt
# model.fit(Xs, ys)

# Find model coefficients that are >0.001 and count
# nfea1 = 


# Sort the coeffients in decreasing order and print. 
# sorted_coeff = np.argsort( ... )


#### Question:
Observe the coefficient values carefully. Do they make sense to you? Do you see positive coefficients for features that you think  are likely to increase student performance, and vice verse? Put your answer below. You may need to consult the original datasource, to understand the meaning of each chosen feature.


#### Type answer here:


# Determine linear predictor with the selected features using the optimal alpha

Note that we cannot use the coefficients determined above as is as our predictor, as it is fitted using all data. Also it obtained by minimizing the LASSO loss. With the selected features, we now go through a 10 fold cross validation to determine the predictor coefficients and the estimated test error using the **linear regression** directly.

First get new feature array only containing these chosen features and name it *X_new*. Print a few lines to make sure that you got the data correctly.

Note: You can convert the dataframe into numpy array using `np.array(...)`

In [ ]:
# TO DO
# X_new = 



Now compute the mean predictor coefficients and mean test error using 10 fold cross validation. For this part, you can use the linear predictor on the original features (not scaled) directly, or using the scaled data. Using the scaled data enables you to judge the importance of featurs based on the coefficient magnitude. Therefore let us use scaled data. 

Store the mean coefficients in `coeff_mean` and the mean intercept in `intercept_mean`. Print them along with the feature names. Also print the mean and std of RSS.

Hint: Get the intercept using `regr.intercept_` attribute.

In [ ]:
# Scale the data
#Xs_new = preprocessing.scale(X_new)

# TODO
# 10 fold CV using the linear regression model to find the RSS and coef using each fold 

# Hint: First set up arrays to store the test errors and coefficients then go through a loop of 10 folds

# RSSts = np.zeros((nfold,1))
# coef=np.zeros((nfold,nfea1))
# intercept=np.zeros((nfold,1))


Using the **mean predictor** above, compute the predicted response variable on the whole data (`Xs_new`). Plot the predicted vs. actual values. Also show a line of 45 degree slope. Also print the normalized RSS.

Hint: Use $\hat{y} = Xs_{new}\beta+\beta_0$, where $\beta$ is the vector `coef_mean` and $\beta_0$ is `intercept_mean`.

In [ ]:
# TO DO
# Compute the response and plot

# yhat = 



# Feature Ranking based on F-test and mutual information 

Rank all the original features (`Xs`) using f-test metric. Print first *nfea1* top ranked feature names. If you wish, you could try other metrics as well (e.g. mutual information). 

In [ ]:
from sklearn.feature_selection import f_regression

# TO DO



### Test Error with f_test
Take the top ranked *nfea1* features, apply linear regression fit in cross validation to find the test error, to see how they compare with LASSO result.

In [ ]:
# TO DO



#### Question:
Comment on the features choosen, their corresponding coefficient values, and the test error, in contrast to those obtained with LASSO. Explain why the feature ranking method is generally not as effective as LASSO.

#### Type answer here: 


### Test error with MI (Optional)
Similarly obtain the test error with MI and compare with LASSO and f_test.
